# Example for settine up time-series processing

**Author**: David Bekaert - Jet Propulsion Laboratory

This notebook provides an example on how one could set up a processing stack and run it through MintPy procesisng. For detailed information on aria-tools one should look into the seperate ariaDownload, ariaExtract, ariaTSprep, and MintPy notebooks seperately.


In this notebook, we give an example on how to process the main island of Hawaii uisng ARIA Geocoded UNWrapped interferogram (GUNW) products using on few user-defined parameters. 

    
<div class="alert alert-warning">
Both the initial setup (<b>Prep A</b> section) and download of the data (<b>Prep B</b> section) should be run at the start of the notebook. All other sections (examples and applications) do not need to be run in order.
</div>

<div class="alert alert-danger">
<b>Potential Errors:</b> 
    
- GDAL uses "HDF5" driver instead of "netCDF/Network Common Data Format" on GUNW products. Verify GDAL version >= 3.
- ARIA-tools and MintPy needs to be installed to run this notebook.
- Virtual data access requires additional pre-requisites as liste on the ARIA-tools website.
</div>


## Table of Contents:
<a id='example_TOC'></a>

[**1. Verifiy if pre-requisites are installed**](#prereq)

[**2. User configurable inputs**](#inputs)

[**3. Download of products - ariaDownload.py**](#download)

[**4. Minimum overlap computation**](#overlap)

[**5. Time-series preparation - ariaTSprep.py**](#tsprep)

[**6. Time-series processing - MintPy**](#mintpy)

## 1. Verifiy if pre-requisites are installed
<a id='prereq'></a>

Below we verify if aria-tools and mintpy are installed on your system.

In [27]:
import os
tutorial_home_dir = os.path.abspath(os.getcwd())
print("Tutorial directory: ", tutorial_home_dir)


# Verifying if ARIA-tools is installed correctly
try:
    import ARIAtools.shapefile_util as shputil
except:
    raise Exception('ARIA-tools is missing from your PYTHONPATH')
 
# Verifying if Mintpy is installed correctly
# verify if mintpy install is complete:
try:
    import numpy as np
    from mintpy import view, tsview, plot_network, plot_transection, plot_coherence_matrix
except:
    print("Looks like mintPy is not fully installed")
    

Tutorial directory:  /Users/dbekaert/Software/ARIA_tools/ARIA-tools-docs-clone/JupyterDocs/ariaEnd2End
Looks like mintPy is not fully installed


## 2. User configurable inputs
<a id='inputs'></a>

Below, are a minum of configurable parameters.

### 2.1 Workdirectory

The working directory in which the outputs are organized in separate subdirectories.

In [28]:
work_dir = os.path.abspath(os.getcwd())

### 2.2 Bounding Box

The user can specify a rectangular bounding box (South-North-West-East coordinates) ` 'S N W E'` or can provide a more complex area of interest using a shapefile or geoJSON as ` path_to_file.shp`. A schematic example is shown in **Fig 1** for both scenarios.

<img src="./support_docs/crop.png" alt="cropping" width="700">
<blockquote><center><b> Fig. 1 </b> Schematic examples of the user-specified area of interest. </center></blockquote>

Below we include two scenario's for a use to pick from:

In [29]:
bbox_SNWE = "18.8 20.3 -156.1 -154.8"
bbox_shp = os.path.join(tutorial_home_dir, 'support_docs', 'Big_Island', 'Big_Island.shp')

# Set the bbox to be used in subsequent processing:
bbox = bbox_SNWE

### 2.3 S1 track number 

Sentinel-1 track number (string) for which the data will be downloaded

In [30]:
tracknumber = '124'

### 2.4 Start and end time of Sentinel-1 period

Start and end time of the Sentinel-1 record provided in YYYYMMDD string format.

In [31]:
start = '20181215'
end = '20190101'

### 2.5 Download format

If your machine support virtual data access, see [here](https://github.com/aria-tools/ARIA-tools#aria-tools-with-support-for-s3-virtual-data-access) for the requirements, one can opt to avoid data-download from S3.

In [32]:
# True/False for using virtual data download
virtualDownload = False


# no changes needed below
# Selecting the download option to be used in subsequent processing.
if virtualDownload:
    download = 'url'
    downloadDir = os.path.join(work_dir,'URLproducts')
else:
    download = 'download'
    downloadDir = os.path.join(work_dir,'products')
    

### 2.6 Summary of user inputs:

In [33]:
print("Work directory: ", work_dir)
print("Download: ", download)
print("Start: ", start)
print("End: ", end)
print("Track: ", tracknumber)
print("bbox: ", bbox)

Work directory:  /Users/dbekaert/Software/ARIA_tools/ARIA-tools-docs-clone/JupyterDocs/ariaEnd2End
Download:  download
Start:  20181215
End:  20190101
Track:  124
bbox:  18.8 20.3 -156.1 -154.8


## 3. Download of products
<a id='download'></a>

Will use ariaDownload to download the products. Netcdf products will be downloaded unless the virtual data option was selected, which downloads a list of product URL locations in S3..

In [74]:
!ariaDownload.py -t {tracknumber} -w {downloadDir} -b "{bbox}" -s {start} -e {end} -o {download}

https://api.daac.asf.alaska.edu/services/search/param?asfplatform=Sentinel-1%20Interferogram%20(BETA)&processingLevel=GUNW_STD&output=JSON&relativeOrbit=124&bbox=-156.1,18.8,-154.8,20.3
 > attempting to download https://urs.earthdata.nasa.gov/profile
 > Re-using previous cookie jar.
 > Download file S1-GUNW-A-R-124-tops-20181222_20181216-043022-21160N_19087N-PP-20dd-v2_0_1.nc exists! 
 > Skipping download of https://grfn.asf.alaska.edu/door/download/S1-GUNW-A-R-124-tops-20181222_20181216-043022-21160N_19087N-PP-20dd-v2_0_1.nc. 
 > Download file S1-GUNW-A-R-124-tops-20181222_20181216-042955-19662N_17420N-PP-52b3-v2_0_1.nc exists! 
 > Skipping download of https://grfn.asf.alaska.edu/door/download/S1-GUNW-A-R-124-tops-20181222_20181216-042955-19662N_17420N-PP-52b3-v2_0_1.nc. 


Download Summary 
--------------------------------------------------------------------------------
  Successes: 0 files, 0 bytes 
  Skipped: 2 files
          - https://grfn.asf.alaska.edu/door/download/S1-GUNW-A-R

## 4. Minimum overlap computation
<a id='overlap'></a>

The minimum area threshold can be used to reject those inteferferograms for which the intersection with the bounding box does not meet a minimum area overlap as set by the user. As long as the user study-area is within one of the ARIA production AOI's, there should always be a temporally conencted network, although partial intererograms can exist when the user study-area covers multiple ARIA production AOI's.

We will enforce that interferograms needs to compeltely cover the study area of the user. To do so we will calculate what this area will be in advsance and then specify it as a threshold in subsequent processing. 

We will apply the following steps:
- We will estiamte the ground swath by using historic SLC data for the given track and bounding box. We will leverage for this the DAAC api to retreive SLC granule bounding boxes and take the union of these as indicator for the ground track.
- We will intersect the user boundingbox with this ground track to compute and calculate its area to estiamte the  spatial coverage of Sentinel-1 within the users study area.


### Estimating the S1 ground swath

First we will qeuerry the DAAC api for historic data and retrieve a CSV file with meta-data.

In [43]:
## Caling the DAAC API and retrieving the SLCs outlines
# checking if bbox exist
from ARIAtools.shapefile_util import open_shapefile
if os.path.exists(os.path.abspath(bbox)):
    bounds = open_shapefile(bbox, 0, 0).bounds
    W,S,E,N = [str(i) for i in bounds]
else:
    try:
        S, N, W, E = bbox.split()
    except:
        raise Exception('Cannot understand the --bbox argument. Input string was entered incorrectly or path does not exist.')

url_base    = 'https://api.daac.asf.alaska.edu/services/search/param?'
url = '{}platform=SENTINEL-1&processinglevel=SLC&beamSwath=IW&output=CSV&maxResults=5000000'.format(url_base)
url += '&relativeOrbit={}'.format(tracknumber)
url += '&bbox=' + ','.join([W,S,E,N])

# could also include start and end time for period. Needs to be of the form:
#start=2018-12-15T00:00:00.000Z&end=2019-01-01T23:00:00.000Z

url = url.replace(' ', '+')
print (url)

!wget -O test.csv "{url}"

https://api.daac.asf.alaska.edu/services/search/param?platform=SENTINEL-1&processinglevel=SLC&beamSwath=IW&output=CSV&maxResults=5000000&relativeOrbit=124&bbox=-156.1,18.8,-154.8,20.3
--2020-07-21 07:03:00--  https://api.daac.asf.alaska.edu/services/search/param?platform=SENTINEL-1&processinglevel=SLC&beamSwath=IW&output=CSV&maxResults=5000000&relativeOrbit=124&bbox=-156.1,18.8,-154.8,20.3
Resolving api.daac.asf.alaska.edu (api.daac.asf.alaska.edu)... 3.215.210.44, 52.72.162.97
Connecting to api.daac.asf.alaska.edu (api.daac.asf.alaska.edu)|3.215.210.44|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: ‘test.csv’

test.csv                [         <=>        ] 351.00K   112KB/s    in 3.1s    

2020-07-21 07:03:03 (112 KB/s) - ‘test.csv’ saved [359419]



Next we will load the meta-data and merge all the SLC granules to estimate the ground swath

In [46]:
import pandas as pd
from ARIAtools.shapefile_util import shapefile_area
from shapely.geometry import Polygon


track_metadata=pd.read_csv('test.csv',index_col=False)

# Compute polygons
SLCPolygons=[]
for frameNdx,frame in track_metadata.iterrows():
    # Convert frame coords to polygon
    SLCPolygons.append(polygonFromFrame(frame))

# Find union of polygons
swathPolygon=SLCPolygons[0]
for SLCPolygon in SLCPolygons:
    swathPolygon=swathPolygon.union(SLCPolygon)
print(swathPolygon)


def polygonFromFrame(frame):
    '''
        Create a Shapely polygon from the coordinates of a frame.
    '''
    P=Polygon([(float(frame['Near Start Lon']),float(frame['Near Start Lat'])),
                (float(frame['Far Start Lon']),float(frame['Far Start Lat'])),
                (float(frame['Far End Lon']),float(frame['Far End Lat'])),
                (float(frame['Near End Lon']),float(frame['Near End Lat']))])
    return P



POLYGON ((-154.0474148747312 17.68374737840119, -154.047348 17.683403, -154.0473518656033 17.68340228646121, -154.047272 17.682991, -156.375031 17.253328, -156.3750763901162 17.25354629054082, -156.375793 17.253414, -156.3757938533037 17.25341810323907, -156.375854 17.253407, -156.375862802599 17.25344932866857, -156.37587 17.253448, -156.3758811336548 17.2535015390008, -156.37648 17.253391, -156.3766157427139 17.25404376540238, -156.377167 17.253942, -156.3772166273392 17.25418063774645, -156.377258 17.254173, -156.3772665568825 17.25421414264429, -156.377289 17.25421, -156.3773128818507 17.2543248524389, -156.37735 17.254318, -156.3774161487335 17.25463606181115, -156.377579 17.254606, -156.5162124922642 17.92120737578276, -156.51622 17.921206, -156.8528715016432 19.53636836768914, -156.8754851546709 19.64421692597727, -156.875763 19.644167, -157.066757 20.555923, -157.0666552026215 20.55594109695356, -157.182022 21.106146, -157.1813678743961 21.10626151749542, -157.181412 21.106472,

### Estimating the minimum overlap by intersection of the ground track and user boundingbox

In [56]:
# TODO convert the bbox in a shapefile (SNWG option), or load the shapefile (shapefile option)
swathPolygon
bboxPolygon = swathPolygon

# TODO use shapely intersect function.
bboxSwathPolygon=bboxPolygon.intersection(swathPolygon)

# TODO calculate shapely area.
minOverlap = shapefile_area(bboxSwathPolygon)
print("Common intersection has an area of %fkm\u00b2"%(minOverlap))


# TODO for numerical issue, should make this area threshold a little bit smaller. i.e. 90% or so
minOverlap = minOverlap*0.9
print("Minimum Area threshold set to 90%" + " or %fkm\u00b2"%(minOverlap))

Common intersection has an area of 118873.002094km²
Minimum Area threshold set to 90% or 106985.701884km²


## 5. Time-series preparation
<a id='tsprep'></a>

Next we will use ariaTSprep to prepapre for time-series processing. This includes the stitching of individual products, and extraction of the inputs expected by MintPy. 

In [57]:
!ariaTSsetup.py -f "{downloadDir}/*" -b "{bbox}" --mask Download --dem Download -mo {minOverlap}

***Time-series Preparation Function:***
Shapefile ./user_bbox.json created for input user bounds.
Multi-core version
/Users/dbekaert/Software/ARIA_tools/ARIA-tools-docs-clone/JupyterDocs/ariaEnd2End/products/download_products_124track_0.txt is not a supported file type... skipping
/Users/dbekaert/Software/ARIA_tools/ARIA-tools-docs-clone/JupyterDocs/ariaEnd2End/products/download_products_124track_1.txt is not a supported file type... skipping
Multi-core version failed, will try single for loop
/Users/dbekaert/Software/ARIA_tools/ARIA-tools-docs-clone/JupyterDocs/ariaEnd2End/products/download_products_124track_0.txt is not a supported file type... skipping
/Users/dbekaert/Software/ARIA_tools/ARIA-tools-docs-clone/JupyterDocs/ariaEnd2End/products/download_products_124track_1.txt is not a supported file type... skipping
3 out of 5 GUNW products rejected for not meeting user's bbox spatial criteria
Group GUNW products into spatiotemporally continuous interferograms.
All (1) interferograms 

Three VRT files *cohStack.vrt*, *connCompStack.vrt*, *unwrapStack.vrt* have been generated under the *stack* subdirectory of your specified work directory. They point to your extracted coherence, connected component, and unwrapped phase files, respectively. The relevant meta-data is directly embedded within these vrt stacks. Note that these VRT stack files are virtual files pointing to individual interferograms in the *unwrappedPhase* and *coherence* directories. In addition, a DEM file (*SRTM_3arcsec.dem*) and water mask (*watermask.msk*) have been downloaded in respectively the *dem* and *mask* folders

In [76]:
!ls stack//*
!ls dem/SRTM_3arcsec.dem
!ls mask/watermask.msk

stack//cohStack.vrt      stack//connCompStack.vrt stack//unwrapStack.vrt
dem/SRTM_3arcsec.dem
mask/watermask.msk


## 6. Time-series processing 
<a id='mintpy'></a>

In [77]:
!smallbaselineApp.py -t TStemplate.txt --dostep load_data


Traceback (most recent call last):
  File "/Users/dbekaert/Software/Mintpy/PySAR_home/mintpy/objects/resample.py", line 11, in <module>
    import pyresample as pr
ModuleNotFoundError: No module named 'pyresample'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/dbekaert/Software/Mintpy/PySAR_home/mintpy/smallbaselineApp.py", line 21, in <module>
    import mintpy.workflow  #dynamic import for modules used by smallbaselineApp workflow
  File "/Users/dbekaert/Software/Mintpy/PySAR_home/mintpy/workflow/__init__.py", line 36, in <module>
    importlib.import_module(root_module + '.' + module)
  File "/Users/dbekaert/Software/python/miniconda37/envs/ARIA-tools_latest/lib/python3.8/importlib/__init__.py", line 127, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
  File "/Users/dbekaert/Software/Mintpy/PySAR_home/mintpy/geocode.py", line 16, in <module>
    from mintpy.objects.resample imp